# Tworzenie zasobów

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1: Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [1]:
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re

def get_links(root, page):
  res = [x['href'] for x in page.find_all('a', href=True)]
  return [x if re.match(r'^https?:\/\/', x) else urljoin(root, x) for x in res if x != root]

def get_paragraphs(page):
  res = [x.get_text().strip() for x in page.find_all('p')]
  return [x for x in res if x]
  
root = 'https://news.ycombinator.com'
page = BeautifulSoup(urlopen(root).read())
data = get_paragraphs(page)

for url in get_links(root, page)[:5]:
  print('scraping', url)
  subpage = BeautifulSoup(urlopen(url).read())
  data += get_paragraphs(subpage)

print(data)

scraping https://news.ycombinator.com/news
scraping https://news.ycombinator.com/newest
scraping https://news.ycombinator.com/front
scraping https://news.ycombinator.com/newcomments
scraping https://news.ycombinator.com/ask
["I'm not sure about that. HN is moderated very effectively. I couldn't comment on the workload there, not being a moderator.", "I'm leery of what you may be trying to imply here. This isn't some conspiracy thing, is it?", "They've decided not to have a stake in the future of our nation. They've proven themselves incapable of long-term planning beyond their own immediate gratification. They have no skin in the game.", 'Why would we want them to vote? How could we possibly trust them to make responsible decisions with their track record?', "If the devil offered them $10 cash on condition that the world is swallowed in flames in 100 years, what's to stop them?", "This comedy video of Health Secretary Matt Hancok has gone viral, but you can't laugh - it's painful to wa

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory cech dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

Historycznie CONLL był bardzo konkretnym formatem danych w którym mieliśmy z góry narzucone cechy (np. format CONLL-U https://universaldependencies.org/docs/format.html). Liczba cech ewoluowała jednak w czasie i w wielu miejscach CONLL stał się synonimem ogólnego formatu, w którym dobór cech zależy tylko od nas, jednak stałym jest zapis sekwencji tokenów jako sekwencji wierszy w tekście, gdzie każdy wiersz jest listą oddzielonych spacją wartości (cech), a zdania oddzielone są od siebie podwójnym znakiem nowej linii.


### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników), nie trzeba więc zamieniać napisu "NN" na "RZECZOWNIK".


In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize

def generate_conll(text):
  for sentence in sent_tokenize(text):
    for i, word in enumerate(word_tokenize(sentence)):
      tag = pos_tag(word)[0][1]
      is_digit = int(tag == 'CD')
      is_punct = int(tag in ',.')
      if is_punct: tag = '_'
      print(i+1, word, tag, is_digit, is_punct)
    print()

generate_conll("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.")

1 Kate NNP 0 0
2 uses JJ 0 0
3 IPhone PRP 0 0
4 5 CD 1 0
5 and DT 0 0
6 IPad PRP 0 0
7 . _ 0 1

1 Kate NNP 0 0
2 bought NN 0 0
3 2 CD 1 0
4 lolipops NN 0 0
5 . _ 0 1




Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

Zwróćcie Państwo uwagę na ostatnią kolumnę, czytając tekst od góry w dół, wystąpienie literki "B" oznacza początek interesującej frazy (Apple), jeśli zaraz za "B" pojawia się sekwencja oznaczona jako "I" - kolejne tokeny stanowią kontynuację interesującej nas frazy, w tym przypadku 3 tokeny "Apple IPhone 5" tworzą jeden byt. Poza tym widzimy, że "iPad" stanowi osobny, jednotokenowy byt.

Po co rozróżniać pomiędzy "B", "I" i "O", czy nie można uwzględnić tylko dwóch tagów "wewnątrz frazy", "poza frazą"? Teoretycznie można, ale wprowadzimy w ten sposób sytuacje niejednoznaczne. 

Sprawdźmy to na przykładzie sekwencji "XBox Playstation" reprezentującej 2 osobne byty. Używając tagowania IOB nasza sekwencja wyglądałaby tak:

XBox B
PlayStation B

Widzimy więc, że dwa tagi "B" oznaczają dwa początki osobnych fraz. Co jednak gdybyśmy używali tagów "wewnątrz (interesującej nas) frazy", "poza (interesującą nas) frazą"?

XBox "wewnątrz (interesującej nas) frazy"
Playstation "wewnątrz (interesującej nas) frazy"

W tej sytuacji oznaczyliśmy poprawnie oba tokeny jako części interesujących nas fraz. Jednak nie wiemy, czy XBox Playstation to jedna, czy dwie osobne frazy (byty) -- stąd format IOB jest zdecydowanie bezpieczniejszym wyborem.

**Zadanie**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
def generate_CONLL(text, devices=[]):
  for sentence in nlp(text).sents:
    for i, token in enumerate(sentence):

      iob = 'O'
      if devices:
        begin, end = devices[0]['begin'], devices[0]['end']
        if begin == token.idx:
            iob = 'B'
        elif begin < token.idx < end:
            iob = 'I'
        if end <= token.idx + len(token):
          devices.pop(0)
  
      is_digit, is_punct = int(token.is_digit), int(token.is_punct)
      pos = '_' if is_digit or is_punct else token.pos_
      print(i+1, token, pos, is_digit, is_punct, iob)
    print()
    
generate_CONLL("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.", devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}])

1 Kate PROPN 0 0 O
2 uses VERB 0 0 O
3 IPhone PROPN 0 0 B
4 5 _ 1 0 I
5 and CCONJ 0 0 O
6 IPad PROPN 0 0 B
7 . _ 0 1 O

1 Kate PROPN 0 0 O
2 bought VERB 0 0 O
3 2 _ 1 0 O
4 lolipops NOUN 0 0 O
5 . _ 0 1 O



Często chcemy w tekście naraz oznaczać byty, które należą do różnych kategorii, np. lokacje, numery telefonów, daty, wzmianki o osobach. W takich sytuacjach używa się również kodowania IOB jednak wzbogaca się etykiety o odpowiednie informacje używając formatu:

{tag IOB}-{etykieta kategorii}

Stąd daty przyjmują oznaczenia: B-DATE / I-DATE, osoby B-PERSON / I-PERSON, numery telefonów B-PHONENUMBER / I-PHONENUMBER, lokacje: B-LOCATION / I-LOCATION itp. Wiemy zatem czy dany token należy do interesującej nas frazy i do jakiej kategorii przypisana jest ta fraza.